In [1]:
import numpy as np
import pandas as pd

# get data
# ! conda install pandas-datareader s
#import pandas_datareader as pdr

# visual
# ! pip install mpl-finance
#import matplotlib.pyplot as plt
#import mpl_finance as mpf

#import seaborn as sns

#time
from datetime import datetime
from datetime import timedelta

# https://github.com/mrjbq7/ta-lib
# ! pip install ta-lib
#import talib

import jieba
import jieba.analyse
jieba.set_dictionary('dict.idkrsi.txt')     # 改預設字典
jieba.analyse.set_stop_words("stopword.goatwang.kang.txt") #指定stopwords字典

In [2]:
df_bbs = pd.read_csv("bda2019_dataset/bbs2.csv",encoding="utf-8")
df_forum = pd.read_csv("bda2019_dataset/forum2.csv",encoding="utf-8")
df_news = pd.read_csv("bda2019_dataset/news2.csv",encoding="utf-8")
df_news['comment_count']=0

df_article = pd.concat([df_forum, df_bbs, df_news])  #三個合併
del df_bbs, df_forum, df_news

df_article['post_time'] = pd.to_datetime(df_article['post_time'])
df_article['post_time2'] = df_article['post_time'].dt.date   # .dt.date用在dataframe  .date()用在一個 #只留日期
#df_article['label'] = 'even'
df_article['content'] = df_article['content'].astype(str).str.replace(',' , ' ').str.replace('\n' , ' ').str.replace('"' , ' ').str.replace("'" , ' ')
df_article['title'] = df_article['title'].astype(str).str.replace(',' , ' ').str.replace('\n' , ' ').str.replace('"' , ' ').str.replace("'" , ' ')
df_article = df_article.sort_values(by=['post_time']).reset_index(drop=True) # 用post_time排序 # 在重設index
df_article.head(2)
#df_article2 = df_article[['post_time2','title','content']]

/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2
0,鉅亨網,0,鉅亨網編譯郭照青這一年，獲利真是難上難。<BR>根據CNBC分析報導，興業銀行的數據顯示，2...,1451580107540_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-01 00:06:00,重大要聞,yahoo股市,淪落的2015 竟是78年來最難獲利的一年,2016-01-01
1,Bokolo,10,1.原文連結(必須檢附)： http://magazine.chinatimes.com/m...,1451580710262_PTT02R,bbs,http://www.ptt.cc/bbs/Stock/M.1451580509.A.722...,2016-01-01 00:48:25,Stock,Ptt,[新聞] 第三方支付 台灣普及關鍵年,2016-01-01


In [3]:
df_TWSE2018 = pd.read_csv("bda2019_dataset/TWSE2018.csv",encoding="utf-8")
df_TWSE2017 = pd.read_csv("bda2019_dataset/TWSE2017.csv",encoding="utf-8")
df_TWSE2016 = pd.read_csv("bda2019_dataset/TWSE2016.csv",encoding="utf-8")

df_TWSE = pd.concat([df_TWSE2016, df_TWSE2017, df_TWSE2018])  #三年合併
del df_TWSE2016, df_TWSE2017, df_TWSE2018

# ['開盤價(元)', '最高價(元)', '最低價(元)', '收盤價(元)', '成交量(千股)', '成交值(千元)', '成交筆數(筆)', '流通在外股數(千股)', '本益比-TSE', '股價淨值比-TSE']
df_TWSE['證券代碼'] = df_TWSE['證券代碼'].astype(str)
df_TWSE['年月日'] = pd.to_datetime(df_TWSE['年月日'])
df_TWSE['開盤價(元)'] = df_TWSE['開盤價(元)'].str.replace(',' , '').astype('float64') # 1,000 to 1000 to float
df_TWSE['最高價(元)'] = df_TWSE['最高價(元)'].str.replace(',' , '').astype('float64')
df_TWSE['最低價(元)'] = df_TWSE['最低價(元)'].str.replace(',' , '').astype('float64')
df_TWSE['收盤價(元)'] = df_TWSE['收盤價(元)'].str.replace(',' , '').astype('float64')
df_TWSE['成交量(千股)'] = df_TWSE['成交量(千股)'].str.replace(',' , '').astype('float64')
df_TWSE['成交值(千元)'] = df_TWSE['成交值(千元)'].str.replace(',' , '').astype('float64')
df_TWSE['成交筆數(筆)'] = df_TWSE['成交筆數(筆)'].str.replace(',' , '').astype('int64')
df_TWSE['流通在外股數(千股)'] = df_TWSE['流通在外股數(千股)'].str.replace(',' , '').astype('float64')
df_TWSE['本益比-TSE'] = df_TWSE['本益比-TSE'].str.replace(',' , '').astype('float64')
df_TWSE['股價淨值比-TSE'] = df_TWSE['股價淨值比-TSE'].astype('float64')
df_TWSE.head(2) 

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE
0,1101 台泥,2016-12-30,29.09,29.64,28.93,29.64,6121.0,213121.0,2939,3692175.0,20.56,1.23
1,1102 亞泥,2016-12-30,24.91,24.91,24.67,24.81,3347.0,88078.0,1227,3361447.0,31.75,0.73


In [4]:
# 選那家股票
company_name = '國巨'
#文章包含那家字
company_words = '國巨'
#company_words = '國巨|被動元件|積層陶瓷電容|MLCC|電感|晶片電阻|車用電子|凱美|同欣電|大毅|君耀|普斯'

In [5]:
# 用日期排序 再把index重排
#2327 
#df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains('國巨')].sort_values(by=['年月日']).reset_index(drop=True)
#2456 
#df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains('奇力新')].sort_values(by=['年月日']).reset_index(drop=True)
#2478 
#df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains('大毅')].sort_values(by=['年月日']).reset_index(drop=True)
#6271 
#df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains('同欣電')].sort_values(by=['年月日']).reset_index(drop=True)

df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains(company_name)].sort_values(by=['年月日']).reset_index(drop=True)
del df_TWSE
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE
0,2327 國巨,2016-01-04,68.75,68.75,67.33,68.62,1128.0,59378.0,697,651594.0,9.38,1.37
1,2327 國巨,2016-01-05,68.10,68.49,67.33,68.36,1067.0,55785.0,703,651594.0,9.34,1.36


In [6]:
##增欄位:fluctuation幅度 tag漲跌平
df_trend['fluctuation'] = 0.0
df_trend['tag']='--'
df_trend['closeshift'] = 0.0
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag,closeshift
0,2327 國巨,2016-01-04,68.75,68.75,67.33,68.62,1128.0,59378.0,697,651594.0,9.38,1.37,0.0,--,0.0
1,2327 國巨,2016-01-05,68.10,68.49,67.33,68.36,1067.0,55785.0,703,651594.0,9.34,1.36,0.0,--,0.0


In [7]:
# ##增欄位:fluctuation幅度 tag漲跌平
# df_trend['fluctuation'] = 0.0
# df_trend['tag']='--'

# ###計算漲跌
# for index, row in df_trend.iterrows():
#     try:
#         margin =(float(df_trend.loc[index,'收盤價(元)']) - float(df_trend.loc[index-1,'收盤價(元)']) )/ float(df_trend.loc[index-1,'收盤價(元)'])
#         df_trend.loc[index,'fluctuation']=margin
#         if margin >=0.03:
#             df_trend.loc[index,'tag']='up'
#         elif margin <= -0.03:
#             df_trend.loc[index,'tag']='down'
#         else:
#             df_trend.loc[index,'tag']='even'
#     except:
#         continue

In [8]:
df_trend['closeshift'] = df_trend['收盤價(元)'].shift(periods=1)#.fillna(value=0.0, inplace=True)
#df_trend['closeshift'].fillna(value= 0.0, inplace=True)
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag,closeshift
0,2327 國巨,2016-01-04,68.75,68.75,67.33,68.62,1128.0,59378.0,697,651594.0,9.38,1.37,0.0,--,NaN
1,2327 國巨,2016-01-05,68.10,68.49,67.33,68.36,1067.0,55785.0,703,651594.0,9.34,1.36,0.0,--,68.62


In [9]:
df_trend['fluctuation'] = (df_trend['收盤價(元)'] - df_trend['closeshift']) / df_trend['closeshift']
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag,closeshift
0,2327 國巨,2016-01-04,68.75,68.75,67.33,68.62,1128.0,59378.0,697,651594.0,9.38,1.37,NaN,--,NaN
1,2327 國巨,2016-01-05,68.10,68.49,67.33,68.36,1067.0,55785.0,703,651594.0,9.34,1.36,-0.003789,--,68.62


In [10]:
df_trend['tag']='even'
df_trend.loc[df_trend['fluctuation'] >=  0.02, 'tag'] = 'up'
df_trend.loc[df_trend['fluctuation'] <= -0.02, 'tag'] = 'down'
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag,closeshift
0,2327 國巨,2016-01-04,68.75,68.75,67.33,68.62,1128.0,59378.0,697,651594.0,9.38,1.37,NaN,even,NaN
1,2327 國巨,2016-01-05,68.10,68.49,67.33,68.36,1067.0,55785.0,703,651594.0,9.34,1.36,-0.003789,even,68.62


In [11]:
up_trend=df_trend[df_trend['tag']=='up']
len(up_trend)

161

In [12]:
down_trend=df_trend[df_trend['tag']=='down']
len(down_trend)

126

In [13]:
#df_company = df_article[ df_article['content'].str.contains('國巨')]   # df 某欄位 string contains "國巨"
#df_company = df_article[ df_article['content'].str.contains('奇力新')] 
#df_company = df_article[ df_article['content'].str.contains('大毅')]  
#df_company = df_article[ df_article['content'].str.contains('同欣電 ')]
df_company = df_article[ df_article['content'].str.contains(company_words)]   
print(len(df_company))
del df_article
df_company.head(2)

6376


,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2
865,財訊快報,0,◆新台幣連八貶，創80個月新低，昨收33.453元，農曆封關前可能貶破34元。<BR> ◆人...,1452126731305_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-07 07:58:00,重大要聞,yahoo股市,工商時報晨訊,2016-01-07
2458,comjj45,0,◆買超 代碼 股票名稱 買 超 買 進 賣 出 買 進 賣 出 ...,1452872063254_PTT02R,bbs,http://www.ptt.cc/bbs/Stock/M.1452871811.A.2FB...,2016-01-15 23:30:09,Stock,Ptt,[其他] 1/15 台灣集中市場自營商買賣超前30名,2016-01-15


In [14]:
stopwords=list()
with open('stopword.goatwang.kang.txt', 'r',encoding='utf-8') as data:
    for stopword in data:
        stopwords.append(stopword.strip('\n'))

In [16]:
# 'content'全部切詞
corpus = []  # array
for index, row in df_company.iterrows():  
    not_cut = df_company.loc[index,'content']
#     not_cut = row['description']                    # 跟上一行一樣意思
    seg_generator = jieba.cut(not_cut, cut_all=False)  # genarator
    seglist = list(seg_generator)                     # 整篇文章string切出來的list
#    seglist = list(filter(lambda a: a not in stopwords and a != '\n', seglist ))  #去除停用詞 #未必需要這步驟
    corpus.append(' '.join(seglist))          # ' '.join(seg_generator)也可

df_company["content2"]=corpus  

In [17]:
df_company.head(2)

,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2,content2
865,財訊快報,0,◆新台幣連八貶，創80個月新低，昨收33.453元，農曆封關前可能貶破34元。<BR> ◆人...,1452126731305_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-07 07:58:00,重大要聞,yahoo股市,工商時報晨訊,2016-01-07,◆ 新台幣 連八貶 ， 創 80 個 月 新低 ， 昨收 33.453 元 ， 農曆 封關 ...
2458,comjj45,0,◆買超 代碼 股票名稱 買 超 買 進 賣 出 買 進 賣 出 ...,1452872063254_PTT02R,bbs,http://www.ptt.cc/bbs/Stock/M.1452871811.A.2FB...,2016-01-15 23:30:09,Stock,Ptt,[其他] 1/15 台灣集中市場自營商買賣超前30名,2016-01-15,◆ 買超 代碼 股票 名稱 買 超 買 進 賣 ...


In [18]:
df_trend.loc[2,'年月日'].date() +  timedelta(days=-1) == df_trend.loc[1,'年月日'].date()

True

In [19]:
df_trend.loc[5,'年月日'].date() +  timedelta(days=-1) == df_trend.loc[4,'年月日'].date()

False

In [20]:
d = df_trend.loc[1,'年月日'].date() - df_trend.loc[ 1-1 ,'年月日'].date() #相減差幾天
d

datetime.timedelta(1)

In [21]:
d.days #只取天數

1

In [22]:
int(d.days) #幾天 轉整數

1

In [23]:
df_trend.loc[3,'年月日'].date() 

datetime.date(2016, 1, 7)

In [24]:
df_company[ df_company['post_time2'] == df_trend.loc[3,'年月日'].date() ].head() # 某欄位 == n 的 全部撈出來

,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2,content2
865,財訊快報,0,◆新台幣連八貶，創80個月新低，昨收33.453元，農曆封關前可能貶破34元。<BR> ◆人...,1452126731305_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-07 07:58:00,重大要聞,yahoo股市,工商時報晨訊,2016-01-07,◆ 新台幣 連八貶 ， 創 80 個 月 新低 ， 昨收 33.453 元 ， 農曆 封關 ...


In [25]:
# # 演算法 
# for index, row in df_2327.iterrows():
#     try:
#         if df_2327.loc[index,'年月日'].date() +  timedelta(days=-1) == df_2327.loc[index-1,'年月日'].date():                        
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-1), 'label'] = df_2327.loc[index,'tag']
#             # 如果股票前一筆差1天  # 那前1天的文章標上當天的漲跌   
        
#         elif df_2327.loc[index,'年月日'].date() +  timedelta(days=-2) == df_2327.loc[index-1,'年月日'].date():           
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-1), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-2), 'label'] = df_2327.loc[index,'tag']
#             # 如果股票前一筆差2天 #那前2天的文章標上當天的漲跌
            
#         elif df_2327.loc[index,'年月日'].date() +  timedelta(days=-3) == df_2327.loc[index-1,'年月日'].date():
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-1), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-2), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-3), 'label'] = df_2327.loc[index,'tag']
#         elif df_2327.loc[index,'年月日'].date() +  timedelta(days=-4) == df_2327.loc[index-1,'年月日'].date():
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-1), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-2), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-3), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-4), 'label'] = df_2327.loc[index,'tag']


#     except:
#         continue

In [26]:
# 看所有相差的天數
# for index, row in df_2327.iterrows():
#     try:
#         n = df_2327.loc[index,'年月日'].date() - df_2327.loc[index-1,'年月日'].date()
#         print(n)
#     except:
#         continue

# 最多12天

In [27]:
# 如果股票前一筆差n天  # 那前n天的文章標上當天的漲跌 
for index, row in df_trend.iterrows():
    try:
        n = int((df_trend.loc[index,'年月日'].date() - df_trend.loc[index-1,'年月日'].date()).days ) # 差幾個datetime # 轉天數 # 再轉整數
#        print(n)

        for i in range(1, n+1):
#            print(i)
            df_company.loc[df_company['post_time2'] ==  df_trend.loc[index,'年月日'].date() +  timedelta(days=-i), 'label5566'] = df_trend.loc[index,'tag']
    except:
        continue

In [28]:
print(len(df_company[df_company['label5566']=='down']))
df_company[df_company['label5566']=='down'].head(2)

1925


,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2,content2,label5566
17050,Moneydj理財網,0,MoneyDJ新聞 2016-04-21 13:02:47 記者 蔡承啟 報導<BR>全球晶...,1461216622247_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-04-21 13:02:00,重大要聞,yahoo股市,都是日圓的錯！晶片電阻廠KOA營益降3成、本季恐更慘,2016-04-21,MoneyDJ 新聞 2016 - 04 - 21 13 : 02 : 47 記...,down
17072,鉅亨網,0,國安基金宣布退場，並宣示賣紅不賣黑，引發台股上漲恐有壓力，不過台股在電子權值股台積電及鴻海領...,1461222022467_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-04-21 14:45:00,重大要聞,yahoo股市,台股盤後－國安基金賣股沒在怕 台積電鴻海領軍止穩 上漲54點,2016-04-21,國安 基金 宣布 退場 ， 並 宣示 賣紅不賣 黑 ， 引發 台股 上漲 恐有 壓力 ， 不...,down


In [29]:
print(len(df_company[df_company['label5566']=='up']))
df_company[df_company['label5566']=='up'].head(2)

2116


,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2,content2,label5566
17173,dufflin,13,1.原文連結(必須檢附)： http://www.chinatimes.com/realti...,1461283311137_PTT02R,bbs,http://www.ptt.cc/bbs/Stock/M.1461282885.A.684...,2016-04-22 07:54:41,Stock,Ptt,[新聞] 國巨配息又減資 股東每股可拿4元,2016-04-22,1 . 原文 連結 ( 必須 檢附 ) ： http : / / www . china...,up
23332,Yahoo奇摩股市,0,（開盤日15:30出刊）美國股市昨晚因為陣亡將士紀念日休市一天，因此今天台股早盤波動相對小，...,1464680466472_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-05-31 15:23:00,重大要聞,yahoo股市,【Y晚報】調降權重免驚 台股暴量拉尾收平盤,2016-05-31,（ 開盤 日 15 : 30 出刊 ） 美國 股市 昨晚 因為 陣亡 將士 紀念日 休市 一...,up


In [30]:
import re
features = []   # features=list()
with open('finance.words.txt', 'r',encoding='utf-8') as data:
    for line in data:
#        line = re.sub('[a-zA-Z0-9\W]', '', line)                     # 把數字英文去掉
        line = re.sub('[0-9]', '', line)                              # 把數字去掉
        features.append(line.replace('\n', '').replace(' ', ''))      # 空格 \n去掉

print(len(features))
print(type(features))
features[:10]

3405
<class 'list'>


['營收', '交易', '成長', '上漲', '外資', '法人', '增加', '下跌', '獲利', '股']

In [31]:
# df_keyword1 = pd.read_csv("final_higher_tf_idf_part.csv",encoding="utf-8") #上漲形容詞
# df_keyword2 = pd.read_csv("final_lower_tf_idf_part.csv",encoding="utf-8")  #下跌形容詞
# df_keyword = pd.concat([df_keyword1,df_keyword2])
# del df_keyword1,df_keyword2
# df_keyword.head()

In [32]:
# features = df_keyword['key'].to_numpy()
# features

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

#features = [ '上漲','下跌','看好','走高','走低','漲停','跌停']
#features = features[:1000]
#cv = TfidfVectorizer()   #預設有空格就一個feature
cv = TfidfVectorizer(vocabulary = features) # 設定自己要的詞 

r = pd.SparseDataFrame(cv.fit_transform(df_company['content2']), 
                       df_company.index,
                       cv.get_feature_names(), 
                       default_fill_value=0)

r.head(2)

,營收,交易,成長,上漲,外資,法人,增加,下跌,獲利,股,...,跌停價,死魚,抬轎,高殖利率股,不弱,有守,續貶,跌回,轉空,執行率
865,0.320404,0.0,0.170801,0.0,0.068154,0.0,0.0,0.028294,0.061494,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2458,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
df_company2 = pd.concat([df_company,r], axis=1)
df_company2.head(2)

,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,...,跌停價,死魚,抬轎,高殖利率股,不弱,有守,續貶,跌回,轉空,執行率
865,財訊快報,0,◆新台幣連八貶，創80個月新低，昨收33.453元，農曆封關前可能貶破34元。<BR> ◆人...,1452126731305_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-07 07:58:00,重大要聞,yahoo股市,工商時報晨訊,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2458,comjj45,0,◆買超 代碼 股票名稱 買 超 買 進 賣 出 買 進 賣 出 ...,1452872063254_PTT02R,bbs,http://www.ptt.cc/bbs/Stock/M.1452871811.A.2FB...,2016-01-15 23:30:09,Stock,Ptt,[其他] 1/15 台灣集中市場自營商買賣超前30名,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
df_company2 = pd.concat([ df_company2[df_company2['label5566']=='up'], df_company2[df_company2['label5566']=='down'] ]) #只取漲跌
Y = df_company2['label5566']
X = df_company2[features]
X.fillna(value=0.0, inplace=True)
X.head(2)

/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/pandas/core/frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


,營收,交易,成長,上漲,外資,法人,增加,下跌,獲利,股,...,跌停價,死魚,抬轎,高殖利率股,不弱,有守,續貶,跌回,轉空,執行率
17173,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23332,0.0,0.0,0.0,0.0,0.117726,0.043055,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
Y.head(2)

17173    up
23332    up
Name: label5566, dtype: object

In [37]:
X = X.to_numpy()
Y = Y.to_numpy()

In [38]:
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [39]:
model_RandomForest = RandomForestClassifier()
model_RandomForest.fit(X, Y)
model_RandomForest.score(X, Y)

/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9816877010640931

In [40]:
model_DecisionTree = DecisionTreeClassifier()
model_DecisionTree.fit(X, Y)
model_DecisionTree.score(X, Y)

0.9928235585251175

In [41]:
model_KNeighbors = KNeighborsClassifier()
model_KNeighbors.fit(X, Y)
model_KNeighbors.score(X, Y)

0.7364513734224202

In [42]:
model_GaussianNB = GaussianNB()
model_GaussianNB.fit(X, Y)
model_GaussianNB.score(X, Y)

0.7154169759960406

In [43]:
model_LogisticRegression = LogisticRegression()
model_LogisticRegression.fit(X, Y)
model_LogisticRegression.score(X, Y)

/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7701064093046276

In [44]:
model_svc = SVC()
model_svc.fit(X, Y)
model_svc.score(X, Y)

/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.5236327641672853

In [ ]:
#predictions = svc.predict(X_validation)
#print(accuracy_score(Y_validation, predictions))
#print(confusion_matrix(Y_validation, predictions))
#print(classification_report(Y_validation, predictions))